In [1]:
import json, pprint, requests, textwrap, time
host = 'http://150.140.142.92:8998'

In [2]:
#get number of sessions
headers = {'Content-Type': 'application/json'}
r = requests.get(host+'/sessions', headers=headers)
con_var=r.json()['total']
print ("Waiting... active sesssions: ", con_var)
while con_var!=0:
    r = requests.get(host+'/sessions', headers=headers)
    con_var=r.json()['total']
    print ("Waiting... active sesssions: ", con_var)
    time.sleep(60)
print ("Good to go!")

Waiting... active sesssions:  1
Waiting... active sesssions:  1
Waiting... active sesssions:  1
Waiting... active sesssions:  1
Waiting... active sesssions:  1
Waiting... active sesssions:  1
Waiting... active sesssions:  1
Waiting... active sesssions:  1
Waiting... active sesssions:  1
Waiting... active sesssions:  1
Waiting... active sesssions:  0
Good to go!


In [3]:
#create new session 
data = {'kind':'pyspark', \
        'name':'Livy', \
        'conf': {'master': 'spark://administrator-OptiPlex-390:7077'}}
headers = {'Content-Type': 'applcation/json'}

r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{'id': 170,
 'name': 'Livy',
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [4]:
#get session information
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()

{'id': 170,
 'name': 'Livy',
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'idle',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['21/06/04 19:12:55 INFO BlockManagerMasterEndpoint: Registering block manager 150.140.142.92:38047 with 366.3 MB RAM, BlockManagerId(driver, 150.140.142.92, 38047, None)',
  '21/06/04 19:12:55 INFO BlockManagerMaster: Registered BlockManager BlockManagerId(driver, 150.140.142.92, 38047, None)',
  '21/06/04 19:12:55 INFO BlockManager: Initialized BlockManager: BlockManagerId(driver, 150.140.142.92, 38047, None)',
  '21/06/04 19:12:55 INFO StandaloneSchedulerBackend: SchedulerBackend is ready for scheduling beginning after reached minRegisteredResourcesRatio: 0.0',
  '21/06/04 19:12:55 INFO SparkEntries: Spark context finished initialization in 762ms',
  '21/06/04 19:12:55 INFO SparkEntries: Created Spark session.',
  '21/06/04 19:12:58 INFO CoarseGrainedSchedulerBackend$DriverEndpoint: Registered executor 

In [5]:
#show session state, repeat until idle to proceed with code submission
state_url = session_url + '/state'
con_var=".."
while con_var!="idle":
    r = requests.get(state_url, headers=headers)
    con_var = r.json()['state']
    print("Job is %s" % r.json()['state'])
    time.sleep(10)
print ("Good to go!")

Job is idle
Good to go!


In [19]:
#send some python statements for execution
codefile = open('Livy_code.py', 'r')
code = codefile.read()
codefile.close()
data = {
    'kind': 'pyspark',
    'code': textwrap.dedent(code)
}

statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)

In [20]:
#show session state, repeat until idle to proceed with result extraction
state_url = session_url + '/state'
con_var=".."
while con_var!="idle":
    r = requests.get(state_url, headers=headers)
    con_var = r.json()['state']
    print("Job is %s" % r.json()['state'])
    time.sleep(10)
print ("Good to go!")

Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is idle
Good to go!


In [18]:
#get stats from the latest statement in the session
statements_url = session_url + '/statements'
print(statements_url)
r = requests.get(statements_url, headers=headers)
slist = r.json()
last_statement = slist['statements'][slist['total_statements']-1]
if last_statement['progress']==1:
    print ("Job completed")

print("Total Time %s ms" % (last_statement['completed'] - last_statement['started']))

http://150.140.142.92:8998/sessions/170/statements
Job completed
Total Time 6658 ms


In [21]:
#close the session
r = requests.delete(session_url, headers=headers)
pprint.pprint(r.json())

{'msg': 'deleted'}
